In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import random
import matplotlib.pyplot as plt

In [12]:
seed = 2024
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
cudnn.deterministic = True
cudnn.benchmark = False

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device is {device}')

device is cuda


In [13]:
# Data Aumgentation - Train Data
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True,
                                 download=True,
                                 transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False,
                                download=True,
                                transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [15]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           num_workers=2)
validation_loader = torch.utils.data.DataLoader(val_dataset,
                                                batch_size=batch_size,
                                                shuffle=False,
                                                num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat', 'deer',
#            'dog', 'frog', 'horse', 'ship', 'truck')

In [16]:
class Block(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, in_planes,
                               kernel_size=3, stride=stride, padding=1,
                               groups=in_planes, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv2 = nn.Conv2d(in_planes, out_planes,
                               kernel_size=1, stride=1, padding=0,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out

In [17]:
class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [64, (128, 2), 128, (256, 2), 256, (512, 2), 512, 512, 512, 512, 512, (1024, 2), 1024]

    def __init__(self):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32)
        self.linear = nn.Linear(1024, 10)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [18]:
model = MobileNet().to(device)
num_epochs = 50
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3) # L2 regularization 적용
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# cnn weight 개수 출력
total_weights = sum(p.numel() for p in model.parameters() if p.requires_grad and p.dim() > 1)
print("Total weights (excluding biases):", total_weights)

Total weights (excluding biases): 3195328


In [19]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    total = 0
    correct = 0
    train_loss = 0.0

    # train
    model.train()
    print(f'   Epoch [{epoch+1}/{num_epochs}]')
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss /= len(train_loader)
    train_accuracy = 100. * correct / total

    # validation
    model.eval()
    val_loss = 0.0
    val_total = 0
    val_correct = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_loss /= len(validation_loader)
    val_accuracy = 100. * val_correct / val_total

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)


    print(f'Train Loss: {train_loss:.4f}')
    print(f'  Val_Loss: {val_loss:.4f}')
    print(f' Train_acc: {train_accuracy:.2f}%')
    print(f'   Val_acc: {val_accuracy:.2f}%\n')

    scheduler.step()

   Epoch [1/50]
Train Loss: 1.8606
  Val_Loss: 1.6311
 Train_acc: 29.68%
   Val_acc: 39.00%

   Epoch [2/50]
Train Loss: 1.5107
  Val_Loss: 1.4419
 Train_acc: 44.56%
   Val_acc: 48.03%

   Epoch [3/50]
Train Loss: 1.3334
  Val_Loss: 1.3686
 Train_acc: 52.12%
   Val_acc: 51.41%

   Epoch [4/50]
Train Loss: 1.2013
  Val_Loss: 1.2625
 Train_acc: 57.08%
   Val_acc: 57.21%

   Epoch [5/50]
Train Loss: 1.0823
  Val_Loss: 1.0545
 Train_acc: 61.86%
   Val_acc: 63.08%

   Epoch [6/50]
Train Loss: 0.9776
  Val_Loss: 0.9988
 Train_acc: 65.88%
   Val_acc: 65.32%

   Epoch [7/50]
Train Loss: 0.8927
  Val_Loss: 0.9143
 Train_acc: 69.36%
   Val_acc: 68.13%

   Epoch [8/50]
Train Loss: 0.8138
  Val_Loss: 0.8279
 Train_acc: 71.94%
   Val_acc: 72.49%

   Epoch [9/50]
Train Loss: 0.7585
  Val_Loss: 0.7596
 Train_acc: 73.97%
   Val_acc: 74.11%

   Epoch [10/50]
Train Loss: 0.7021
  Val_Loss: 0.7536
 Train_acc: 76.09%
   Val_acc: 74.20%

   Epoch [11/50]
Train Loss: 0.6586
  Val_Loss: 0.6914
 Train_acc: 77

KeyboardInterrupt: 

In [ ]:
# test
correct = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

print(f'Test Accuracy: {100.* correct / len(test_loader.dataset):.2f} %')

In [ ]:
# Loss graph
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend()
plt.show()

# Accuracy graph
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, num_epochs+1), val_accuracies, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy')
plt.legend()
plt.show()